# Iteration 6: add in a count of occupancy.

## Plain English Summary

We modify ACU to include a count of the concurrent number of patients on the ward. This modifies the acute treatment functions in the ASU class.

We carry over the `Experiment` class and the lognormal parameter function from iteration 5.

> Testing Note: Now that we have a measure of occupancy we can modify the extreme value tests looking at infinite acute treatment times.

## Notebook imports

In [1]:
from IPython.display import Markdown, display
import math
from sim_tools.distributions import Lognormal

## LLM Prompt

In [2]:
display(Markdown("inputs/06_iteration.md"))

modify the `AcuteStrokeUnit` class as follows: create a member variable called `occupancy`.  Increment the `occupancy` member variable when a patient arrives to the ASU. Decrement the `occupancy` member variable at the end of the `acute_treatment` functions

output only the AcuteStrokeUnit



## Date and time of input to LLM

> 1515 6th March 2024

## Observations on the LLM Output

* The 4th source listed below is the paper that is being recreated.
* The code appears to have been modified correctly with comments highlighting the changes e.g.
```python
self.occupancy += 1  # Increment occupancy when a patient arrives
yield self.env.timeout(length_of_stay)
print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")
self.occupancy -= 1  # Decrement occupancy at the end of treatment
```

## Sources reported by the LLM

* https://www.researchgate.net/publication/360946800_Improving_emergency_treatment_for_patients_with_acute_stroke_the_PEARS_research_programme_including_the_PASTA_cluster_RCT
* https://journals.sagepub.com/doi/pdf/10.1177/23814683221134098
* http://essay.utwente.nl/90598/1/Nijs_BA_BMS.pdf
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5043535/
* https://www.researchgate.net/publication/369679747_Stressed_systems_Stroke_unit_bed_occupancy_and_impact_on_reperfusion_therapy_in_acute_ischemic_stroke

## Code carried over from iteration 5

In [3]:
import simpy
import numpy as np

# External function to convert Lognormal moments to Normal moments
def normal_moments_from_lognormal(mean, std_dev):
    phi = np.sqrt(std_dev ** 2 + mean ** 2)
    mu = np.log(mean ** 2 / np.sqrt(std_dev ** 2 + mean ** 2))
    sigma = np.sqrt(np.log(phi ** 2 / mean ** 2))
    return mu, sigma

In [4]:
# Define the Experiment class to store model parameters
class Experiment:
    def __init__(self, stroke_mean=1.2, tia_mean=9.3, neuro_mean=3.6, other_mean=3.2,
                 rehab_mean=7.4, rehab_std_dev=8.6, esd_mean=4.6, esd_std_dev=4.8,
                 other_dest_mean=7.0, other_dest_std_dev=8.7,
                 tia_dest_mean=1.8, tia_dest_std_dev=5.0,
                 neuro_dest_mean=4.0, neuro_dest_std_dev=5.0,
                 other_dest_mean_2=3.8, other_dest_std_dev_2=5.2,
                 run_length=1825):  # 5 years in days
        self.stroke_interarrival_mean = stroke_mean
        self.tia_interarrival_mean = tia_mean
        self.neuro_interarrival_mean = neuro_mean
        self.other_interarrival_mean = other_mean
        self.rehab_mean = rehab_mean
        self.rehab_std_dev = rehab_std_dev
        self.esd_mean = esd_mean
        self.esd_std_dev = esd_std_dev
        self.other_dest_mean = other_dest_mean
        self.other_dest_std_dev = other_dest_std_dev
        self.tia_dest_mean = tia_dest_mean
        self.tia_dest_std_dev = tia_dest_std_dev
        self.neuro_dest_mean = neuro_dest_mean
        self.neuro_dest_std_dev = neuro_dest_std_dev
        self.other_dest_mean_2 = other_dest_mean_2
        self.other_dest_std_dev_2 = other_dest_std_dev_2
        self.run_length = run_length

## LLM Output

In [5]:
# Define the AcuteStrokeUnit class with occupancy tracking
class AcuteStrokeUnit:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.occupancy = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            mu, sigma = normal_moments_from_lognormal(self.experiment.rehab_mean, self.experiment.rehab_std_dev)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        elif patient_type == "ESD":
            mu, sigma = normal_moments_from_lognormal(self.experiment.esd_mean, self.experiment.esd_std_dev)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        else:
            mu, sigma = normal_moments_from_lognormal(self.experiment.other_dest_mean, self.experiment.other_dest_std_dev)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        self.occupancy += 1  # Increment occupancy when a patient arrives
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")
        self.occupancy -= 1  # Decrement occupancy at the end of treatment

    def tia_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(self.experiment.tia_dest_mean, self.experiment.tia_dest_std_dev)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        self.occupancy += 1  # Increment occupancy when a patient arrives
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")
        self.occupancy -= 1  # Decrement occupancy at the end of treatment

    def neuro_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(self.experiment.neuro_dest_mean, self.experiment.neuro_dest_std_dev)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        self.occupancy += 1  # Increment occupancy when a patient arrives
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")
        self.occupancy -= 1  # Decrement occupancy at the end of treatment

    def other_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(self.experiment.other_dest_mean_2, self.experiment.other_dest_std_dev_2)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        self.occupancy += 1  # Increment occupancy when a patient arrives
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")
        self.occupancy -= 1  # Decrement occupancy at the end of treatment

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.experiment.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            self.env.process(self.stroke_acute_treatment(post_asu_destination))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.experiment.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.experiment.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.experiment.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())


## Run script carried over from iter 5

In [6]:
# Create the simulation environment and Experiment instance with default parameters
env = simpy.Environment()

# print out simulated trace...
default_experiment_params = Experiment()
acu_experiment = AcuteStrokeUnit(env, default_experiment_params)

# Start the patient generators for each type of patient
env.process(acu_experiment.stroke_patient_generator())
env.process(acu_experiment.tia_patient_generator())
env.process(acu_experiment.neuro_patient_generator())
env.process(acu_experiment.other_patient_generator())

# Run the simulation until the specified run length in the Experiment parameters
env.run(until=default_experiment_params.run_length)

Stroke patient 1 arrived at 0.3933744986764033 days and will go to Other
Stroke patient 2 arrived at 0.5107221468798179 days and will go to Other
Stroke patient 3 arrived at 1.1654884702404178 days and will go to Other
Other patient 4 arrived at 1.4435904976712481 days
Stroke patient 5 arrived at 2.3858268464048447 days and will go to Other
Complex Neuro patient 6 arrived at 2.6205051660135874 days
Other patient 6 finished treatment at 3.3006732709551567 days
Complex Neuro patient 6 finished treatment at 3.578614594295762 days
Stroke patient 7 arrived at 3.609443853335145 days and will go to Rehab
Other patient 8 arrived at 3.77863471036749 days
Complex Neuro patient 9 arrived at 3.940782773178247 days
Stroke patient 10 arrived at 4.556037871551747 days and will go to Rehab
Stroke patient 11 arrived at 4.835135346210568 days and will go to Other
Stroke patient 11 finished treatment at 5.207410727976926 days
Stroke patient 12 arrived at 5.261316487310658 days and will go to ESD
Complex 

## Testing
* **Testing lognormal function**: Test that lognomal function correctly calculates the moments of the underlying normal dist.
    * Expected result: The LLM generated function and `sim-tools.distributions.Lognormal` perform the same conversion.
    * Actual results (PASS):  The functions generate the same values.
* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 10 simulated time periods
    * Actual result (PASS): The model runs no longer than 10 simulated time periods
* **Extreme value test 3**: All patient types have have their mean length of stay time set to $M$ a very large number
    * Expected result: No patients depart the model (only arrivals seen in log)
    * Actual Results (PASS): No patients depart the model.
* **Extreme value test 4**: All patient types apart from stroke->rehab have have their mean length of stay time set to $M$ a very large number
    * Expected result: Only stroke patients depart the model (seen in log)
    * Actual Results (PASS): Only stroke patient depart the model.

### Lognormal test

Test that lognomal function correctly calculates the moments of the underlying normal dist.

**Result: PASS**

In [7]:
def test_lognormal_moments(mean, std):
    '''
    Test that lognomal function correctly calculates 
    the moments of the underlying normal dist.

    Params:
    ------
    mean: float
        mean of the lognormal distribution

    std: float
        st dev of the lognormal distribution

    Returns:
    -------
    bool
    '''
   
    # Lognormal class from sim-tools.
    expected_moments = Lognormal(mean, std)
    print(expected_moments.mu, expected_moments.sigma)
    
    ## llm code
    llm_mu, llm_sigma = normal_moments_from_lognormal(mean, std)
    print(llm_mu, llm_sigma)
    
    return (llm_mu, llm_sigma) == (expected_moments.mu, expected_moments.sigma)

In [8]:
test_lognormal_moments(128.79, 267.51)

4.022977459259615 1.2924439729790718
4.022977459259615 1.2924439729790718


True

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [9]:
M = 10_000_000

In [10]:
def test_1(large_number):
    '''
    TIA, Complex Neuro, Other, have their inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model 
    is "Stroke". This is seen in the event log

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        tia_mean=large_number,
        neuro_mean=large_number,
        other_mean=large_number,
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    return acu_experiment.patient_count

In [11]:
test_1(M)

Stroke patient 1 arrived at 0.280711272095134 days and will go to ESD
Stroke patient 2 arrived at 1.115813738679098 days and will go to Other
Stroke patient 3 arrived at 1.2283205307090344 days and will go to Other
Stroke patient 4 arrived at 1.3366858839835525 days and will go to Rehab
Stroke patient 4 finished treatment at 2.7934396337229925 days
Stroke patient 4 finished treatment at 3.823342751928095 days
Stroke patient 4 finished treatment at 4.077618703547181 days
Stroke patient 5 arrived at 4.20737149865872 days and will go to Other
Stroke patient 6 arrived at 4.461624953669774 days and will go to Other
Stroke patient 6 finished treatment at 4.943169300433787 days
Stroke patient 7 arrived at 6.383104580875904 days and will go to Other
Stroke patient 7 finished treatment at 8.159246026116145 days
Stroke patient 7 finished treatment at 8.213004718483441 days
Stroke patient 7 finished treatment at 9.410564061598713 days
Stroke patient 8 arrived at 9.494257984217136 days and will go

1509

### Extreme value test 2

* An experiment is setup so that all patient types have a mean inter-arrival rate of $M$

**Result: PASS**

In [12]:
def test_2(large_number):
    '''
    All patient types have have their inter-arrival time is 
    set to $M$ a very large number
    
    Expected result: No patients arrive to the model.
    
    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        stroke_mean=large_number,
        tia_mean=large_number,
        neuro_mean=large_number,
        other_mean=large_number,
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    return acu_experiment.patient_count

In [13]:
test_2(M)

0

### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [14]:
TEN_DAYS = 10

In [15]:
def test_3(run_length):
    '''
    Test that the run length of the model can be modified.
    
    Expected result: The model should runs no longer than the new run length.

    Params:
    ------
    run_length: float
        Run length of the model in days.

    Returns:
    -------
    float
        Simulation end time.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        run_length=run_length,
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    return env.now

In [16]:
test_3(TEN_DAYS)

Stroke patient 1 arrived at 0.8312061275742753 days and will go to Rehab
Stroke patient 2 arrived at 1.5279934474145138 days and will go to Other
Complex Neuro patient 3 arrived at 1.801948213974646 days
Stroke patient 4 arrived at 2.00032634929446 days and will go to Other
Other patient 5 arrived at 2.0099669624032765 days
Other patient 6 arrived at 2.278961051683846 days
Other patient 7 arrived at 2.3523376362963835 days
Other patient 7 finished treatment at 2.953214762464876 days
Stroke patient 8 arrived at 3.0713324650907303 days and will go to Other
Complex Neuro patient 9 arrived at 3.179223322393296 days
Stroke patient 10 arrived at 3.4150668259631765 days and will go to ESD
Stroke patient 10 finished treatment at 3.4703305986677844 days
Other patient 10 finished treatment at 4.178427553053783 days
Other patient 11 arrived at 4.200197111953786 days
Other patient 12 arrived at 4.504654749646 days
Stroke patient 12 finished treatment at 4.662794684935884 days
Stroke patient 12 fin

10

### Extreme value test 3

* An experiment is setup so that all patient types have a mean length of stay on the ASU of $M$

**Result: PASS**

In [17]:
def test_4(large_number):
    '''
    All patient types have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: No patients depart the model 
    The number of arrivals = the occupancy of the model.

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        rehab_mean=M,  # stroke->rehab
        esd_mean=M,    # stroke-> ESD
        other_dest_mean=M, # stroke->other
        tia_dest_mean=M,   # TIA patients
        neuro_dest_mean=M, # complex neuro patients
        other_dest_mean_2=M, # other (med outlier) patients
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    print(f'{acu_experiment.patient_count=}')
    print(f'{acu_experiment.occupancy=}')
    
    return acu_experiment.patient_count == acu_experiment.occupancy

In [18]:
result = test_4(M)

Stroke patient 1 arrived at 0.5362000040306819 days and will go to Other
Stroke patient 2 arrived at 0.8815819943706152 days and will go to Other
Stroke patient 3 arrived at 0.9542696028479098 days and will go to Other
Stroke patient 4 arrived at 0.9989198841102275 days and will go to Other
Stroke patient 5 arrived at 1.400678801234468 days and will go to Other
Stroke patient 6 arrived at 1.7057157526643314 days and will go to Rehab
Other patient 7 arrived at 2.339313219796481 days
Stroke patient 8 arrived at 3.7715694158342727 days and will go to Other
Stroke patient 9 arrived at 4.018812490827493 days and will go to Other
Stroke patient 10 arrived at 4.023471582001913 days and will go to Other
Stroke patient 11 arrived at 4.631888397331688 days and will go to Other
Stroke patient 12 arrived at 5.6216566842386015 days and will go to Other
Stroke patient 13 arrived at 6.347592700846766 days and will go to ESD
Complex Neuro patient 14 arrived at 7.255436948900926 days
TIA patient 15 arr

In [19]:
result

True

### Extreme value test 4

* An experiment is setup so that all patient types other than stroke->rehab have a mean length of stay on the ASU of $M$

**Result: PASS**

In [20]:
def test_5(large_number):
    '''
    All patient types apart from stroke-rehab patients
    have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: Only stroke patients depart the
    model.
    (assed by viewing  log and patient_count > occupancy)

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        rehab_mean=7.4,  # stroke->rehab
        esd_mean=M,    # stroke-> ESD
        other_dest_mean=M, # stroke->other
        tia_dest_mean=M,   # TIA patients
        neuro_dest_mean=M, # complex neuro patients
        other_dest_mean_2=M, # other (med outlier) patients
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    print(f'{acu_experiment.patient_count=}')
    print(f'{acu_experiment.occupancy=}')
    
    return acu_experiment.patient_count > acu_experiment.occupancy

In [21]:
results = test_5(M)

TIA patient 1 arrived at 0.31680270756172274 days
Other patient 2 arrived at 0.8342966179241027 days
Stroke patient 3 arrived at 1.7919445200409816 days and will go to Other
Complex Neuro patient 4 arrived at 2.0336449570280872 days
Stroke patient 5 arrived at 2.11137112239058 days and will go to ESD
Stroke patient 6 arrived at 3.3938660674933896 days and will go to Other
Stroke patient 7 arrived at 4.169295965951667 days and will go to Other
Stroke patient 8 arrived at 4.853320418922288 days and will go to Other
Stroke patient 9 arrived at 8.093786902656197 days and will go to Other
Stroke patient 10 arrived at 9.598155430038972 days and will go to Other
Stroke patient 11 arrived at 10.182587220079194 days and will go to Rehab
Stroke patient 12 arrived at 11.420078471955105 days and will go to Rehab
Stroke patient 13 arrived at 11.558390855469945 days and will go to Rehab
Stroke patient 14 arrived at 12.760734515756806 days and will go to Other
Stroke patient 15 arrived at 13.21605557

In [22]:
results

True